In [152]:
import os
import torch
import torch.utils.data
import torchvision
from pycocotools.coco import COCO
from coco.PythonAPI.pycocotools.coco import COCO
from PIL import Image

In [345]:
%load_ext blackcellmagic
import torch
import os
import structlog
from datetime import datetime
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
from torchmetrics.functional import accuracy, auroc #,f1
#from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
%config InlineBackend.figure_format='retina'
import os
import glob
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import random
import pytorch_lightning as pl
from tqdm import tqdm
from coco.PythonAPI.pycocotools.coco import COCO
from pycocotools.coco import COCO

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [346]:
import torchvision

### Convert to COCO from data.py file

In [347]:
def _crop_bbox(bbox, width, height):
    """
    It crops the bbox according to width and height maxima, if the bbox is over-sized.
    Additionally it also corrects negative coordinates by setting it to zero.

    bbox coordinates are [[x1,y1][x2,y2]]
    The (x1, y1) position is at the top left corner,
    the (x2, y2) position is at the bottom right corner
    """

    # check for over-sized bbox
    margin = 2  # px
    bbox[0] = width - margin if bbox[0] > width else bbox[0]
    bbox[1] = height - margin if bbox[1] > height else bbox[1]
    bbox[2] = width - margin if bbox[2] > width else bbox[2]
    bbox[3] = height - margin if bbox[3] > height else bbox[3]

    # correct negative coordinates
    bbox = [c if c >= 0 else 0.0 for c in bbox]

    return bbox

In [348]:
DEFAULT_BBOX_SIZE_THRESHOLD = 5

In [349]:
def convert_bbox_XYWH(bbox):
    width = bbox[2] - bbox[0]
    height = bbox[3] - bbox[1]
    return [bbox[0], bbox[1], width, height]

In [350]:
logger = structlog.getLogger(__name__)

In [351]:
from datetime import datetime
def convert_to_coco(annotations, categories):
    """
    Converts the Workist DLA annotations into the Microsoft COCO annotations format.
    More information about the COCO format here: https://cocodataset.org/#home
    :param annotations: annotations in Workist format
    :param categories: list of categories
    :return: COCO-formatted annotations consisting of multiple sections:
    [{
      "info":{
         "description":"Document layout analysis data set",
         "url":"https://www.workist.com",
         "version":"1.0",
         "year":2021,
         "contributor":"Workist GmbH",
         "date_created":"2021/02/09"
      },
      "licenses":[
         {
            "url":"https://www.workist.com",
            "id":1,
            "name":"Workist GmbH Property"
         },...],
      "categories":[
         {
            "supercategory":"orders",
            "id":0,
            "name":"table"
         },...],
      "images":[
         {
            "license":1,
            "file_name":"absolut_path/test.png",
            "height":2339,
            "width":1654,
            "id":0
         },...],
      "annotations":{
         "width":1600,
         "height":2400,
         "url":"images/test.png",
         "file_name":"test/images/test.png",
         "annotations":[
            {
               "class":1,
               "bbox":[
                  -50.0,
                  100.0,
                  1000.0,
                  1100.0
               ],
               "bbox_mode":0,
               "category_id":1
            },...]
      }
    },...]
    """
    # Prepare info section
    info = {
        "description": "Document layout analysis training data set",
        "url": "https://www.workist.com",
        "version": "1.0",
        "year": datetime.utcnow().year,
        "contributor": "Workist GmbH",
        "date_created": datetime.utcnow().date().isoformat(),
    }

    # Prepare license section
    licenses = [{"url": "https://www.workist.com", "id": 1, "name": "Workist GmbH Property"}]

    # Prepare category section
    coco_categories = []
    class_to_id_map = {c: k for k, c in enumerate(categories)}
    for category in categories:
        cat_dict = {"supercategory": "orders", "id": class_to_id_map[category], "name": category}
        coco_categories.append(cat_dict)

    # Prepare image and annotation sections
    coco_images = []
    coco_annotations = []
    jobs_empty_annotations = []
    tiny_bboxes = []
    annotation_id_offset = 0
    for page_count, page in enumerate(annotations):

        # Get pages with no annotation
        if not page["annotations"]:
            jobs_empty_annotations.append(page["operator_job_id"])
            continue

        width, height = page["width"], page["height"]
        image = {"license": 1, "file_name": page["file_name"], "height": height, "width": width, "id": page_count}
        coco_images.append(image)

        # Iterate over all annotations of one page
        for annotation in page["annotations"]:

            # Check and crop bbox for negative and too big coordinates
            annotation["bbox"] = _crop_bbox(annotation["bbox"], width, height)

            # Check if bbox height is tiny (<DEFAULT_BBOX_SIZE_THRESHOLD px)
            if annotation["bbox"][3] - annotation["bbox"][1] < DEFAULT_BBOX_SIZE_THRESHOLD:
                tiny_bboxes.append(page["operator_job_id"])
                continue
            # Check if bbox width is tiny (<DEFAULT_BBOX_SIZE_THRESHOLD px)
            if annotation["bbox"][2] - annotation["bbox"][0] < DEFAULT_BBOX_SIZE_THRESHOLD:
                tiny_bboxes.append(page["operator_job_id"])
                continue

            # Convert bbox with [x1, y1, x2, y2] to [x1, y1, width, height]
            bbox_XYWH = convert_bbox_XYWH(annotation["bbox"])
            coco_annotation = {
                "area": bbox_XYWH[2] * bbox_XYWH[3],
                "iscrowd": 0,
                "image_id": page_count,
                "bbox": bbox_XYWH,
                "category_id": class_to_id_map[categories[annotation["class"]]],
                "id": len(coco_annotations) + 1,
            }
            coco_annotations.append(coco_annotation)

    logger.info(f"Total of {len(jobs_empty_annotations)} pages skipped as no annotations were provided.")
    logger.info(
        f"Total of {len(tiny_bboxes)} annotations skipped as size was invalid (<{DEFAULT_BBOX_SIZE_THRESHOLD}px)."
    )
    logger.info(f"Total of {len(coco_annotations)} annotations converted to the COCO format.")

    # Put everything together
    coco_annotation_json = {
        "info": info,
        "licenses": licenses,
        "categories": coco_categories,
        "images": coco_images,
        "annotations": coco_annotations,
    }
    return coco_annotation_json


### Load data and convert to COCO

In [352]:
import json

In [353]:
data_dir = "/Users/saranobrega/Desktop/2022-11-22_object_detection_data"
annotations='/Users/saranobrega/Desktop/2022-11-22_object_detection_data/all_annotations.json'
categories='/Users/saranobrega/Desktop/2022-11-22_object_detection_data/categories.json'

In [354]:
annotations

'/Users/saranobrega/Desktop/2022-11-22_object_detection_data/all_annotations.json'

In [355]:
#with open('/Users/saranobrega/Desktop/2022-11-22_object_detection_data/all_annotations.json') as fp:
#    annotations_data = json.load(fp)

In [356]:
type(annotations_data)

list

In [357]:
print(len(annotations_data))

65935


In [358]:
#annotations_data

In [359]:
#with open('/Users/saranobrega/Desktop/2022-11-22_object_detection_data/categories.json') as fp:
#    categories_data = json.load(fp)

In [360]:
type(categories_data)

list

In [361]:
categories_data

['table', 'footer', 'header', 'order_line_item_header', 'order_line_item']

Subsampling

In [362]:
def create_reduced_dataset_dir(src_dir, dst_dir, data_perc):
    
    '''
    src_dir: path to 022-11-22_object_detection_data directory
    dst_dir: path to new dir with n number of samples
    data_perc: percentage of the original dataset (number of samples in the reduced dataset)
    '''
    
    src_dir = str(src_dir) + '/2022-11-22_object_detection_data'
    
    with open(src_dir + '/all_annotations.json', 'r') as file:
         all_annotations = json.load(file)

    with open(src_dir + '/categories.json', 'r') as file:
         categories = json.load(file)
    
    annotations_len = len(all_annotations)
    sample_size = round(annotations_len*data_perc)
    n_random_annotations = random.sample(all_annotations, sample_size)
    

    #path to the new directory
    dst_dir = str(dst_dir) + '/object_detection_data_' + str(sample_size) + '_samples' 

    #create dst_dir if it doesn't exist
    if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

    #make images dir inside dst_dir
    images_dir = dst_dir + '/images'
    if not os.path.exists(images_dir):
            os.makedirs(images_dir)

    #copy images
    for i in range(0,len(n_random_annotations)):
        image_name = n_random_annotations[i]['file_name']
        shutil.copy(os.path.join(src_dir, image_name), images_dir)

    #save sampled annotations and categories
    with open(os.path.join(dst_dir,'all_annotation.json'),'w') as json_file:
             json.dump(n_random_annotations, json_file)

    with open(os.path.join(dst_dir,'categories.json'),'w') as json_file:
             json.dump(categories, json_file)

In [1]:
create_reduced_dataset_dir(src_dir='/Users/saranobrega/Desktop',
                           dst_dir='/Users/saranobrega/Desktop',
                           data_perc=0.05)

Convert to COCO

In [365]:
#data_COCO['annotations']

In [366]:
#data_COCO['images']

In [338]:
#annotations_only=data_COCO['annotations']

In [340]:
len(annotations_only

4685

In [105]:
type(data_COCO)

dict

In [110]:
#print(data_COCO.items())
# doesnt work

In [113]:
data_COCO_list = list(data_COCO.items())
#data_COCO_list 

In [111]:
#dict format
#with open('data_coco_format_dict.json', 'w', encoding='utf-8') as f:
#    json.dump(data_COCO, f)

In [115]:
#list format
#with open('data_coco_format_list.json', 'w', encoding='utf-8') as f:
#    json.dump(data_COCO_list, f)

In [94]:
# only annotations
#annotations_only=data_COCO['annotations']

In [147]:
#first_value = list(data_COCO.values())[0:3]
#first_value

In [ ]:
# trying test split

In [309]:
#data_COCO

In [313]:
annotations=data_COCO['annotations']

In [367]:
#annotations

In [317]:
categories=data_COCO['categories']

In [368]:
#categories

In [310]:
data_COCO.keys()

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])

In [312]:
list(data_COCO)

['info', 'licenses', 'categories', 'images', 'annotations']

In [201]:
train_df, val_df = train_test_split(df, test_size=0.05,shuffle=True)
train_df.shape, val_df.shape

((631636, 2), (33244, 2))

In [ ]:
train_df = train_df.reset_index()
val_df = val_df.reset_index()


### Dataset validator to verify and check if the dataset is in COCO format:

https://www.section.io/engineering-education/understanding-coco-dataset/

In [49]:
coco_data=data_COCO

In [66]:
def assertions(key, values, required_keys, unique_key=None):
    unique_key_id_mapper = {} # Mapping to keep track of the images
    for value in values: # Looping through each key to map the unique key with 'id'
        if unique_key is not None:
            unique_key_id_mapper[value['id']] = value[unique_key] # Map the 'unique_key' with 'id'
    for required_key in required_keys:
        assert required_key in value, "'{}' does not contain the required key '{}'".format(key, required_key) # Check if the required key is present in parent object
    return unique_key_id_mapper # Return the mapping

In [72]:
def annotation_assertions(key, annotations, image_map, category_map):
    required_keys = ['area', 'iscrowd', 'bbox', 'category_id', 'image_id', 'id'] # Specify all the required keys under 'annotations'
    assertions('annotations', coco_data['annotations'], required_keys, None) # Assert the presence of all the required keys within the parent object
    for annotation in annotations:
        assert len(annotation['bbox']) == 4, "'{}' key in 'annotations' does not match the expected format".format('bbox') # Check if 'bbox' contains a list of 4 elements
        assert annotation['category_id'] in category_map, "'{}' is not present in the 'categories' mapping".format('category_id') # Check if the 'category_id' is present in the mapping that we created earlier
        assert annotation['image_id'] in image_map, "'{}' is not present in the 'images' mapping".format('image_id') # Check if the 'image_id' is present in the mapping that we created earlier
        assert annotation['area'] == (annotation['bbox'][2] * annotation['bbox'][3]), "Mismatch of values in '{}' and '{}'".format('area', 'bbox') # Check 'area' is a product of 'width' and 'height'
        #assert len(annotation['segmentation'][0]) == 8 or len(annotation['segmentation']) == 0, "'{}' must either be an empty list or contain a list of 8 values".format('segmentation') # Check if the 'segmentation' values contains expected list
        assert annotation['iscrowd'] == 0 or annotation['iscrowd'] == 1, "'{}' must either be 0 or 1. {} is invalid".format('iscrowd', annotation['iscrowd']) # Check if 'iscrowd' contains either '0' or '1'


In [73]:
def main():
    required_keys = ['images', 'annotations', 'categories'] # Have a copy for the mandatory keys in a list
    for required_key in required_keys: # Loop through each required key
        assert required_key in coco_data.keys(), "Required key '{}' not found in the COCO dataset".format(required_key) # Check if the required key is present
        assert len(coco_data[required_key]) > 0, "Required key '{}' does not contain values".format(required_key) # Check if the required key contains value

    image_map = assertions('images', coco_data['images'], ["file_name", "height", "width", "id"], "file_name") # Verify the presence of keys present under 'images'
    category_map = assertions('categories', coco_data['categories'], ["id", "name", "supercategory"], "name") # Verify the presence of keys present under 'categories'
    annotation_assertions('annotations', coco_data['annotations'], image_map, category_map) # Verify the validity of keys under annotations
    print('The dataset format is COCO!') # If no error is thrown, the dataset is validated to be in COCO format

if __name__ == '__main__':
    main()

The dataset format is COCO!
